In [1]:
import sqlalchemy
import json
import datetime
import warnings
import pickle
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem.snowball import SnowballStemmer

In [2]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        stemmer = SnowballStemmer("english", ignore_stopwords=True)
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

In [17]:
#Predict Team Name,component and classification based on model
def predict_TeamName_Component_Classification(defect_summary=''):
    #connect models to determine based on text summary
    #Load Models
    predictions ={}
    model_Team_Name = pickle.load(
        open(
             os.getcwd()+"/outputs/Team Name_SVM_RS.model", "rb"
        )
    )
    model_Component = pickle.load(
        open(
             os.getcwd()+"/outputs/Component_SVM_RS.model", "rb"
        )
    )
    model_Classification = pickle.load(
        open(
             os.getcwd()+"/outputs/Classification_SVM_RS.model", "rb"
        )
    )
    
    pred_Team_Name = model_Team_Name.predict(defect_summary)[0]
    pred_Component = model_Component.predict(defect_summary)[0]
    pred_Classification =model_Classification.predict(defect_summary)[0]
    

    
    predictions = {'Team_Name':pred_Team_Name, 'Component':pred_Component, 'Classification':pred_Classification }
    
    return(predictions)

#issue_summary='Sync error while signing in to profile'
#issue_summary='details not appearing on locked device'
#issue_summary='Crash on toggle of settings on iPad'
#issue_summary='Unable to save credentials for email'
#issue_summary='Browser keeps crashing on opening multiple tabs' 
#issue_summary='Themes resulting in modified colors on the menubar'
issue_summary = 'The application consumes too much memory and throws out of memory error'
Team_Name = predict_TeamName_Component_Classification([issue_summary])['Team_Name']
Component = predict_TeamName_Component_Classification([issue_summary])['Component']
Classification = predict_TeamName_Component_Classification([issue_summary])['Classification']

print("Predicted Team Name :"+Team_Name)
print("Predicted Component :"+Component)
print("Predicted Classification :"+Classification)

Predicted Team Name :Telemetry
Predicted Component :General
Predicted Classification :Components


In [24]:
#Sentiment Analysis based on Vader
def get_sentiment(input_text):
    import nltk
    nltk.download('vader_lexicon')
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sentiment = SentimentIntensityAnalyzer()
    print("Sentiment Score ",sentiment.polarity_scores(input_text)['compound'])
    return(sentiment.polarity_scores(input_text)['compound'])

#Determine negative sentiment in order to send to manager based on vader and DF sentiments.
def detect_user_neg_sentiment(df_sentiment=0, vader_sentiment=0):
    #Send to manager if Dialogflow and custom analysis both suggest really negative sentiment and low CSI
    if (df_sentiment < -0.8) and (vader_sentiment <=-0.7): 
        return(True)
    else:
        return(False)



#vader_sentiment=get_sentiment('The application consumes too much memory and throws out of memory errors')
#vader_sentiment=get_sentiment('The application has really good performance, is light weight but would have been better to have a mobile version as well')
#vader_sentiment=get_sentiment('Rendering in ipad and phone is not so good. However works well in desktop')
df_sentiment=-0.8 #Value from DialogFlow Natural Language API
print(vader_sentiment)

print(detect_user_neg_sentiment(df_sentiment, vader_sentiment))

Sentiment Score  -0.2331
-0.2331
False


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
